In [1]:
from typing import Tuple, List, Union, Any, Optional, Dict, Literal, Callable, Type
import abc

from tqdm import tqdm
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim
import torch.utils.data
from torch import Tensor
from sklearn.linear_model import RidgeClassifierCV
import xgboost as xgb

from models.ridge_ALOOCV import fit_ridge_ALOOCV
from models.sandwiched_least_squares import sandwiched_LS_dense, sandwiched_LS_diag, sandwiched_LS_scalar

In [9]:
import cProfile
import pstats
from models.models import GreedyRFBoostRegressor
import torch
import torch.nn.functional as F

# Make regression data X, y
device = "cpu"
N = 5000
N_test = 5000
D = 10
d = 1
X = torch.randn(N, D).to(device)
X_test = torch.randn(N_test, D).to(device)
w_true = torch.randn(D, d).to(device)
y = (X @ w_true)**2 + torch.randn(N, d).to(device) * 0.1  # Adding some noise
y_test = (X_test @ w_true)**2 + torch.randn(N_test, d).to(device) * 0.1  # Adding some noise


def train_and_evaluate():
    #dense      
    model = GreedyRFBoostRegressor(
         hidden_dim=128, 
         randfeat_x0_dim=512, 
         randfeat_xt_dim=512,
         n_layers=10, 
         l2_reg_sandwich=0.1, 
         feature_type="SWIM", 
         upscale="dense", 
         ).to(device)

    results = []
    for i in range(10):
        model.fit(X, y)
        out = model(X)
        out_test = model(X_test)
        rmse = torch.sqrt(F.mse_loss(out, y))
        rmse_test = torch.sqrt(F.mse_loss(out_test, y_test))
        results.append(torch.tensor([rmse, rmse_test]))
    results = torch.stack(results)
    print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
    print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
    print("train", results[:, 0])
    print("test", results[:, 1])

# Profile the training and evaluation
profiler = cProfile.Profile()
profiler.enable()
train_and_evaluate()
profiler.disable()

# Print profiling results
stats = pstats.Stats(profiler).sort_stats('tottime')
stats.print_stats()

train rmse tensor(0.4533) std tensor(0.0485)
test rmse tensor(0.8308) std tensor(0.0688)
train tensor([0.4579, 0.4375, 0.3467, 0.4724, 0.5238, 0.5132, 0.4564, 0.4403, 0.4329,
        0.4520])
test tensor([0.8463, 0.8267, 0.7113, 0.8684, 0.9321, 0.9309, 0.8111, 0.8027, 0.7600,
        0.8186])
         78584 function calls (73138 primitive calls) in 27.846 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      310   10.179    0.033   10.179    0.033 {built-in method torch._C._linalg.linalg_svd}
      100    4.528    0.045   14.810    0.148 /home/nikita/Code/random-feature-boosting/models/sandwiched_least_squares.py:61(sandwiched_LS_dense)
       20    2.710    0.135    6.560    0.328 /home/nikita/Code/random-feature-boosting/models/models.py:774(forward)
      630    2.485    0.004    2.485    0.004 {built-in method torch._C._nn.linear}
      200    1.910    0.010    2.319    0.012 /home/nikita/Code/random-feature-boosting/mo

In [3]:
import cProfile
import pstats
from models.models import GradientRFBoostRegressor
import torch
import torch.nn.functional as F

# Make regression data X, y
device = "cpu"
N = 5000
N_test = 5000
D = 10
d = 1
X = torch.randn(N, D).to(device)
X_test = torch.randn(N_test, D).to(device)
w_true = torch.randn(D, d).to(device)
y = (X @ w_true)**2 + torch.randn(N, d).to(device) * 0.1  # Adding some noise
y_test = (X_test @ w_true)**2 + torch.randn(N_test, d).to(device) * 0.1  # Adding some noise


def train_and_evaluate():
    #dense      
    model = GradientRFBoostRegressor(
         hidden_dim=128, 
         randfeat_x0_dim=512, 
         randfeat_xt_dim=512,
         n_layers=10, 
         feature_type="SWIM", 
         upscale="dense", 
         ).to(device)

    results = []
    for i in range(5):
        model.fit(X, y)
        out = model(X)
        out_test = model(X_test)
        rmse = torch.sqrt(F.mse_loss(out, y))
        rmse_test = torch.sqrt(F.mse_loss(out_test, y_test))
        results.append(torch.tensor([rmse, rmse_test]))
    results = torch.stack(results)
    print("train rmse", results[:, 0].mean(), "std", results[:, 0].std())
    print("test rmse", results[:, 1].mean(), "std", results[:, 1].std())
    print("train", results[:, 0])
    print("test", results[:, 1])

# Profile the training and evaluation
profiler = cProfile.Profile()
profiler.enable()
train_and_evaluate()
profiler.disable()

# Print profiling results
stats = pstats.Stats(profiler).sort_stats('tottime')
stats.print_stats()

train rmse tensor(0.7411) std tensor(0.0532)
test rmse tensor(1.1036) std tensor(0.0804)
train tensor([0.7805, 0.7528, 0.7104, 0.7960, 0.6659])
test tensor([1.0827, 1.1378, 1.0387, 1.2262, 1.0326])
         41758 function calls (39022 primitive calls) in 19.487 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      105    6.253    0.060   13.803    0.131 /home/nikita/Code/random-feature-boosting/models/ridge_ALOOCV.py:7(fit_ridge_ALOOCV)
      105    4.960    0.047    4.960    0.047 {built-in method torch._C._linalg.linalg_svd}
      105    1.747    0.017    1.747    0.017 {built-in method torch.tensordot}
      315    1.221    0.004    1.221    0.004 {built-in method torch._C._nn.linear}
      100    0.914    0.009    1.134    0.011 /home/nikita/Code/random-feature-boosting/models/models.py:159(fit)
       10    0.884    0.088    2.780    0.278 /home/nikita/Code/random-feature-boosting/models/models.py:999(forward)
      15